# 查询小区的坐标，并对无查询结果的 

已经有的密钥：
+ 登录名：blue_liberty@qq.com  
  邮箱：blue_liberty@qq.com  
  手机号：13708041691  
  密钥：42Tm8e7Khmsi3IuTB15wKCGsejjvZV0e
  
  
+ 登录名：13708041691  
  邮箱：blue_liberty@163.com  
  手机号：18349368961  
  密钥：Rm4oiaWBcithZtuWArB5lPm2oxptk7ig  
  
  
+ 密钥：7QP1yCu8hlfwV3QV1mAhkmpmp0kR2K8d


+ 密钥：2b6XPUZbwNOb8dAGkez2SNfb4hA8p5eV


+ 登录名：15395325356  
  密码：13083035356  
  邮箱：315373812@qq.com  
  密钥：4gYOPlFkxGd5hCXaiK3BQ2oMKUs061TY
  
  
+ 登录名：13675539062  
  密码：13083035356  
  邮箱：13708041691@163.com  
  密钥：EeS0kjMaeFQIoGu2KFRiGaFECqXI1pdi

In [ ]:
# 载入必要的模块
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import json
import requests

In [ ]:
# 读取保存有小区名称的csv文件，将小区名称取唯一值，转换为DataFrame，列名为name
data = pd.read_csv("C:/Users/31537/Downloads/Project/location/freq_over_5.csv", encoding = 'gb18030') # csv文件名需要调整

name = data["sell_name"] # 提取小区名称所在列

name = Series(name)

In [ ]:
# 查看待处理的小区名称的个数
len(name)

In [ ]:
# 创建一个空的DataFrame
result = DataFrame({'name': [], 'location': [], 'address': []})

# 创建一个列名符合要求的空行
empty_txt = DataFrame({'name': [np.nan], 'location': [np.nan], 'address': [np.nan]})

# 选择保留'name', 'address', 'location'三列信息
col = ['name', 'location', 'address']

# 获取小区的经纬度数据
for s in name:
    # 将小区名转化为utf8编码
    s = s.encode("UTF-8")
    s = repr(s).replace('\\x', '%').strip("'")
    
    # 创建API，且设置只保留1行查询结果（page_size=1）
    url = 'http://api.map.baidu.com/place/v2/search?query=' + s + '&page_size=1&page_num=0&scope=1&region=深圳&output=json&ak=4gYOPlFkxGd5hCXaiK3BQ2oMKUs061TY'
    
    resp = requests.get(url)
    data1 = json.loads(resp.text)
    txt = DataFrame(data1['results']).ix[:0, ['name', 'location', 'address']]
    
    # a是查询结果的行数
    a, b = txt.shape
    
    # 判断查询结果是否为空，如果为空（即a为0），则在result后面加上一空行，如果不为空，则在result后面加上查询结果
    if a == 0:
        result = pd.concat([result, empty_txt], axis = 0, ignore_index=True)
    else:
        result = pd.concat([result, txt], axis = 0, ignore_index=True)
    
    # 将查询结果是否为空，显示出来
    print a,

In [ ]:
# 查看获取了经纬度的小区个数
result.shape

In [ ]:
# 预览爬取的小区坐标
result

In [ ]:
# 将result写入名为raw_coordinate的csv文件中
result.to_csv("C:/Users/31537/Downloads/Project/location/raw_coordinate.csv",encoding='gb18030')

In [ ]:
# 处理'location'列，将经度和纬度分离，并写入名为final_coordinate的csv文件中

# 选取location列
location = result['location']

# 创建空列表，后续分离的经纬度将会存放在该列表中
location_result = []

# 将经度和纬度分离，并进行必要的字符串处理
for raw_location in location:
    # 创建一个临时的列表容器
    temp_result = []
        
    # 将经纬度分离
    temp_coordinate = str(raw_location).split(",")
    
    # 进行必要的字符串处理，将结果放入temp_result中
    for loc in temp_coordinate:
        loc = loc.strip("{u'lat': ").strip(" u'lng': ").strip("}")
        temp_result.append(loc)
    
    location_result.append(temp_result)

# 将存放分离的经纬度的列表转换为DataFrame
final_coordinate = DataFrame(location_result)

# 将存放分离的经纬度的DataFrame，写入名为final_coordinate的csv文件中
final_coordinate.to_csv("C:/Users/31537/Downloads/Project/location/final_coordinate.csv",encoding='gb18030')